In [1]:
import pandas as pd
import numpy as np
import os
import re
import pickle


from wordcloud import WordCloud

# libraries to clean the data
import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

# To vectorize and split the data appropriately
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

# To model the Gaussian Navie Bayes classifier
from sklearn.naive_bayes import MultinomialNB

# SVM
from sklearn import svm

# KNN
from sklearn.neighbors import KNeighborsClassifier

# To calculate the accuracy score of the model
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

# scoring models
from sklearn import metrics
from sklearn.metrics import classification_report

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/maanyashanker/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
bill = pd.read_csv("PA/2023-2024_Regular_Session/csv/bills.csv")
docs = pd.read_csv("PA/2023-2024_Regular_Session/csv/documents.csv")
history = pd.read_csv("PA/2023-2024_Regular_Session/csv/history.csv")
people = pd.read_csv("PA/2023-2024_Regular_Session/csv/people.csv")
rollcalls = pd.read_csv("PA/2023-2024_Regular_Session/csv/rollcalls.csv")
sponsors = pd.read_csv("PA/2023-2024_Regular_Session/csv/sponsors.csv")
votes = pd.read_csv("PA/2023-2024_Regular_Session/csv/votes.csv")
# texts = pd.read_csv("bill_texts.csv")

In [3]:
bill.head()

,bill_id,session_id,bill_number,status,status_desc,status_date,title,description,committee_id,committee,last_action_date,last_action,url,state_link
0,1724272,2035,HB1,2,Engrossed,2023-04-25,A Joint Resolution proposing an amendment to t...,A Joint Resolution proposing an amendment to t...,84,Senate State Government,2023-05-08,Referred to STATE GOVERNMENT,https://legiscan.com/PA/bill/HB1/2023,https://www.legis.state.pa.us/cfdocs/billinfo/...
1,1723866,2035,HB2,2,Engrossed,2023-03-07,"In limitation of time, further providing for i...",An Act amending Title 42 (Judiciary and Judici...,77,Senate Judiciary,2023-03-28,Referred to JUDICIARY,https://legiscan.com/PA/bill/HB2/2023,https://www.legis.state.pa.us/cfdocs/billinfo/...
2,1773739,2035,HB19,1,Introduced,2023-07-05,"Further providing for definitions, for respira...","An Act amending the act of December 20, 1985 (...",57,House Professional Licensure,2023-07-05,Referred to PROFESSIONAL LICENSURE,https://legiscan.com/PA/bill/HB19/2023,https://www.legis.state.pa.us/cfdocs/billinfo/...
3,1745716,2035,HB20,1,Introduced,2023-03-23,Providing for registration of extraordinary wa...,An Act providing for registration of extraordi...,45,House Environmental Resources & Energy,2023-03-23,Referred to ENVIRONMENTAL RESOURCES AND ENERGY,https://legiscan.com/PA/bill/HB20/2023,https://www.legis.state.pa.us/cfdocs/billinfo/...
4,1724313,2035,HB21,1,Introduced,2023-03-07,"In table games, further providing for table ga...",An Act amending Title 4 (Amusements) of the Pe...,47,House Finance,2023-03-07,Referred to FINANCE,https://legiscan.com/PA/bill/HB21/2023,https://www.legis.state.pa.us/cfdocs/billinfo/...


Status - Status Number
Passed - 4
Enrolled - 3
Engrossed - 2
Introduced - 1

In [4]:
bill["status_desc"].value_counts()

Introduced    3783
Engrossed      473
Passed         471
Enrolled        32
Name: status_desc, dtype: int64

In [5]:
bill["status"].value_counts()

1    3783
2     473
4     471
3      32
0       3
Name: status, dtype: int64

In [6]:
bill.isnull().sum()

bill_id               0
session_id            0
bill_number           0
status                0
status_desc           3
status_date           3
title                 0
description           0
committee_id          0
committee           926
last_action_date      2
last_action           2
url                   0
state_link            0
dtype: int64

In [7]:
bill[bill['status_desc'].isna()]

,bill_id,session_id,bill_number,status,status_desc,status_date,title,description,committee_id,committee,last_action_date,last_action,url,state_link
2130,1871580,2035,HB2205,0,NaN,NaN,Providing for protection of personal informati...,An Act providing for protection of personal in...,0,NaN,NaN,NaN,https://legiscan.com/PA/bill/HB2205/2023,https://www.legis.state.pa.us/cfdocs/billinfo/...
2948,1871546,2035,HR388,0,NaN,NaN,Directing the Legislative Budget and Finance C...,A Resolution directing the Legislative Budget ...,0,NaN,NaN,NaN,https://legiscan.com/PA/bill/HR388/2023,https://www.legis.state.pa.us/cfdocs/billinfo/...
2972,1877128,2035,HR412,0,NaN,NaN,Amending the General Operating Rules of the Ho...,A Resolution amending the General Operating Ru...,0,NaN,2024-07-01,Laid on the table (Pursuant to House Rule 71),https://legiscan.com/PA/bill/HR412/2023,https://www.legis.state.pa.us/cfdocs/billinfo/...


In [8]:
bill.dropna(subset=['status_desc', 'status_date', 'committee', 'last_action_date', 'last_action'], inplace=True)

In [9]:
len(bill)

3836

In [10]:
df = bill.drop(["status_desc", "committee", "state_link", "url", "bill_number", "session_id", "status_date", "last_action_date"], axis=1)

In [11]:
df.dtypes

bill_id          int64
status           int64
title           object
description     object
committee_id     int64
last_action     object
dtype: object

In [12]:
df = df.astype({'title':'string', 'description':'string', 'last_action':'string'})

In [13]:
df.head()

,bill_id,status,title,description,committee_id,last_action
0,1724272,2,A Joint Resolution proposing an amendment to t...,A Joint Resolution proposing an amendment to t...,84,Referred to STATE GOVERNMENT
1,1723866,2,"In limitation of time, further providing for i...",An Act amending Title 42 (Judiciary and Judici...,77,Referred to JUDICIARY
2,1773739,1,"Further providing for definitions, for respira...","An Act amending the act of December 20, 1985 (...",57,Referred to PROFESSIONAL LICENSURE
3,1745716,1,Providing for registration of extraordinary wa...,An Act providing for registration of extraordi...,45,Referred to ENVIRONMENTAL RESOURCES AND ENERGY
4,1724313,1,"In table games, further providing for table ga...",An Act amending Title 4 (Amusements) of the Pe...,47,Referred to FINANCE


In [14]:
stop_words = stopwords.words('english')

# def sent_to_words(sentences):
#     for sentence in sentences:
#         # deacc=True removes punctuations
#         yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
        
        
def remove_stopwords(texts):
    return [word for word in texts.split() if word not in stop_words]

def clean_text(text):
    text = ' '.join(remove_stopwords(text.lower()))
    # remove punctuation
    text = str(re.sub('[,.!?]', '', text))
    return text


In [15]:
df['clean_title']= df['title'].apply(lambda x:clean_text(x))
df['clean_description']= df['description'].apply(lambda x:clean_text(x))
df['clean_last_action']= df['last_action'].apply(lambda x:clean_text(x))

df.head()

,bill_id,status,title,description,committee_id,last_action,clean_title,clean_description,clean_last_action
0,1724272,2,A Joint Resolution proposing an amendment to t...,A Joint Resolution proposing an amendment to t...,84,Referred to STATE GOVERNMENT,joint resolution proposing amendment constitut...,joint resolution proposing amendment constitut...,referred state government
1,1723866,2,"In limitation of time, further providing for i...",An Act amending Title 42 (Judiciary and Judici...,77,Referred to JUDICIARY,limitation time providing infancy insanity imp...,act amending title 42 (judiciary judicial proc...,referred judiciary
2,1773739,1,"Further providing for definitions, for respira...","An Act amending the act of December 20, 1985 (...",57,Referred to PROFESSIONAL LICENSURE,providing definitions respiratory therapists p...,act amending act december 20 1985 (pl457 no112...,referred professional licensure
3,1745716,1,Providing for registration of extraordinary wa...,An Act providing for registration of extraordi...,45,Referred to ENVIRONMENTAL RESOURCES AND ENERGY,providing registration extraordinary water use...,act providing registration extraordinary water...,referred environmental resources energy
4,1724313,1,"In table games, further providing for table ga...",An Act amending Title 4 (Amusements) of the Pe...,47,Referred to FINANCE,table games providing table game taxes,act amending title 4 (amusements) pennsylvania...,referred finance


In [16]:
X_train_clean_title_text, X_test_clean_title_text, \
X_train_clean_description_text, X_test_clean_description_text, \
X_train_clean_last_action_text, X_test_clean_last_action_text, \
X_train_comm_id, X_test_comm_id, \
target_train, target_test = train_test_split(
    df['clean_title'],
    df['clean_description'],
    df['clean_last_action'],
    df['committee_id'],
    df['status'],
    test_size=0.2,
    stratify=df['status']
)

In [17]:
# TF-IDF Feature Extraction
vectorizer = TfidfVectorizer()
X_train_title = vectorizer.fit_transform(X_train_clean_title_text)
X_test_title = vectorizer.transform(X_test_clean_title_text)
X_train_desc = vectorizer.transform(X_train_clean_description_text)
X_test_desc = vectorizer.transform(X_test_clean_description_text)
X_train_action = vectorizer.transform(X_train_clean_last_action_text)
X_test_action = vectorizer.transform(X_test_clean_last_action_text)

features_train = np.concatenate([X_train_title.todense(), X_train_desc.todense(), X_train_action.todense()], axis=1)
features_test = np.concatenate([X_test_title.todense(), X_test_desc.todense(), X_test_action.todense()], axis=1)

In [23]:
model = MultinomialNB(alpha=3)

model.fit(features_train, target_train)

test_pred = model.predict(features_test)
test_proba = model.predict_proba(features_test)
train_pred = model.predict(features_train)
train_proba = model.predict_proba(features_train)

print("Accuracy:", metrics.accuracy_score(target_test, test_pred))
print("AUC Score:", metrics.roc_auc_score(target_test, test_proba, multi_class='ovr'))
print("Precision:", metrics.precision_score(target_test, test_pred, average='weighted'))
print("Recall:",metrics.recall_score(target_test, test_pred, average='weighted'))
print("F1:", metrics.f1_score(target_test, test_pred, average='weighted'))

# print('Naive Bayes - Train\n', classification_report(target_train, train_pred, zero_division=0))
# print('Naive Bayes - Test\n', classification_report(test_pred, target_test, zero_division=0))

/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


Accuracy: 0.890625
AUC Score: 0.5842888128738603
Precision: 0.793212890625
Recall: 0.890625
F1: 0.8391012396694215


/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-

In [25]:
# instantiate model
model = svm.SVC(kernel='rbf', probability=True) 

# fit model
model.fit(features_train, target_train)

test_pred = model.predict(features_test)
test_proba = model.predict_proba(features_test)
train_pred = model.predict(features_train)
train_proba = model.predict_proba(features_train)

print("Accuracy:", metrics.accuracy_score(target_test, test_pred))
print("AUC Score:", metrics.roc_auc_score(target_test, test_proba, multi_class='ovr'))
print("Precision:", metrics.precision_score(target_test, test_pred, average='weighted'))
print("Recall:",metrics.recall_score(target_test, test_pred, average='weighted'))
print("F1:", metrics.f1_score(target_test, test_pred, average='weighted'))


# print('SVM - Train\n', classification_report(target_train, train_pred, zero_division=0))
# print('SVM - Test\n', classification_report(test_pred, target_test, zero_division=0))

/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-

Accuracy: 0.890625
AUC Score: 0.7007283166839561
Precision: 0.793212890625
Recall: 0.890625
F1: 0.8391012396694215


/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [24]:
# instantiate model
model = KNeighborsClassifier(n_neighbors=11)

# fit model
model.fit(features_train, target_train)

test_pred = model.predict(features_test)
test_proba = model.predict_proba(features_test)
train_pred = model.predict(features_train)
train_proba = model.predict_proba(features_train)

print("Accuracy:", metrics.accuracy_score(target_test, test_pred))
print("AUC Score:", metrics.roc_auc_score(target_test, test_proba, multi_class='ovr'))
print("Precision:", metrics.precision_score(target_test, test_pred, average='weighted'))
print("Recall:",metrics.recall_score(target_test, test_pred, average='weighted'))
print("F1:", metrics.f1_score(target_test, test_pred, average='weighted'))
# false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_score[:,1])


# print('KNN - Train\n', classification_report(target_train, train_pred, zero_division=0))
# print('KNN - Test\n', classification_report(test_pred, target_test, zero_division=0))

/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-

Accuracy: 0.8893229166666666
AUC Score: 0.587281012251171
Precision: 0.8381382426823941
Recall: 0.8893229166666666
F1: 0.8432201109008188


/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
